In [1]:
import pandas as pd
import pyodbc
import datetime
username = 'vigrose'
password = 'Gryffindor~10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [8]:
ppl_query = \
        """
        SELECT DISTINCT
        P.MED_EDU_NBR,
        PC.PARTY_ID,
        P.DEATH_DT
        FROM
        AMAEDW.PERSON P, AMAEDW.PARTY_CAT PC
        WHERE
        PC.CAT_GRP_ID = 450
        AND P.DEATH_DT > '2017-12-31';
        """

In [3]:
sql_query = \
        """
        SELECT DISTINCT
        M.MED_EDU_NBR,
        M.PARTY_ID,
        M.BIRTH_DT,
        M.DEATH_DT,
        P.FIRST_NM,
        P.LAST_NM,
        P.MIDDLE_NM,
        P.LABEL_NM,
        M.BIRTH_CITY_NM
        FROM
        AMAEDW.PERSON M, AMAEDW.PERSON_NM P
        WHERE
        M.PARTY_ID = P.PARTY_ID
        AND
        M.DEATH_DT IS NOT NULL;
        """

In [5]:
DEATH_CODES = pd.read_sql(con=AMAEDW, sql=sql_query)
DEATH_CODES.head()

KeyboardInterrupt: 

In [9]:
DEATH = pd.read_sql(con=AMAEDW, sql=ppl_query)
DEATH.head()

,MED_EDU_NBR,PARTY_ID,DEATH_DT
0,00102030559,1809126,2020-05-16
1,00102460171,1809126,2019-02-23
2,00102470087,1809126,2018-05-07
3,00102470117,1809126,2020-01-30
4,00102500440,1809126,2019-12-26


In [7]:
DEATH.drop_duplicates('MED_EDU_NBR')

,MED_EDU_NBR,PARTY_ID,DEATH_DT
0,00102030559,1809126,2020-05-16
1,00102470117,1809126,2020-01-30
2,00102530021,1809126,2020-05-31
3,00102570057,1809126,2020-11-18
4,00102570111,1809126,2020-11-24
...,...,...,...
4119,95701610017,1809126,2020-06-26
4120,95701910061,1809126,2020-08-22
4121,95702520046,1809126,2020-10-13
4122,95702590036,1809126,2020-09-13


In [14]:
DEATH = DEATH.fillna('None')
DEATH.groupby('CATEGORY_CODE').count()

,PARTY_ID,DEATH_DT
CATEGORY_CODE,,
CAR-ACCID,193560,193560
COVID-19,4703508,4703508
HOMICIDE,193560,193560
NATRL-CAUS,1490412,1490412
SUICIDE,251628,251628


In [ ]:
DEATH_CODES

In [44]:
DEATH.groupby('CATEGORY_CODE').count()

,PARTY_ID
CATEGORY_CODE,
CAR-ACCID,10
COVID-19,147
HOMICIDE,10
NATRL-CAUS,77
SUICIDE,13


In [6]:
PEEPS = pd.read_sql(con=AMAEDW, sql=sql_query)
PEEPS.head()

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,BIRTH_CITY_NM
0,00102000218,1798002,1969-02-13,2015-09-08,Albert,Smith,Lee,Albert Lee Smith MD,BIRMINGHAM
1,00102090063,1798037,1883-11-11,1969-03-28,Amon,Mc Millan,K,Amon K Mc Millan MD,MELVIN
2,00102090063,1798037,1883-11-11,1969-03-28,Amon,Mc Millan,Kennedy,Amon Kennedy Mc Millan,MELVIN
3,00102110013,1798038,1886-10-13,1971-12-21,Percy,Chaudron,Octave,Percy Octave Chaudron MD,MOBILE
4,00102120027,1798039,1888-12-17,1972-10-07,James,Gilchrist,G,James G Gilchrist MD,ALABAMA


In [ ]:
len(PEEPS)

In [7]:
all_death = pd.merge(PEEPS, DEATH, on='PARTY_ID', how='left')

In [53]:
all_death[(all_death['LAST_NM']=='Gunckle')&(all_death['FIRST_NM']=='Wayne')]

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,BIRTH_CITY_NM,CATEGORY_CODE
102940,01875731871,2046578,1947-12-28,2011-03-27,Wayne,Gunckle,M,Wayne M Gunckle DO,DAYTON,NaN


In [20]:
all_death.drop_duplicates('MED_EDU_NBR')

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,CATEGORY_CODE
0,00102000218,1798002,1969-02-13,2015-09-08,Albert,Smith,Lee,Albert Lee Smith MD,NaN
1,00102090063,1798037,1883-11-11,1969-03-28,Amon,Mc Millan,K,Amon K Mc Millan MD,NaN
3,00102110013,1798038,1886-10-13,1971-12-21,Percy,Chaudron,Octave,Percy Octave Chaudron MD,NaN
4,00102120027,1798039,1888-12-17,1972-10-07,James,Gilchrist,G,James G Gilchrist MD,NaN
6,00102140010,1798040,1890-06-24,1976-01-14,William,Clark,Arthur,William Arthur Clark MD,NaN
...,...,...,...,...,...,...,...,...,...
417773,01175190721,12872098,1990-12-06,2018-11-06,Samantha,Kulangara,,Samantha Kulangara,NaN
417774,01175220042,13170249,1994-07-20,2019-08-22,Lindsay,Kightlinger,Marlene,Lindsay Marlene Kightlinger,NaN
417775,02512220498,13294890,1995-12-11,2019-06-25,Annie,Yang,Li,Annie Li Yang,NaN
417776,01104220165,13336133,1994-07-25,2019-03-08,Sarah,Ross,Ellen,Sarah Ellen Ross,NaN


In [25]:
all_death['DEATH_DT'] = pd.to_datetime(all_death.DEATH_DT)

In [27]:
all_death['YEAR']=[x.year for x in all_death.DEATH_DT]

In [37]:
all_death.drop_duplicates('MED_EDU_NBR').groupby('YEAR').count()[20:]

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,CATEGORY_CODE
YEAR,,,,,,,,,
1965,33,33,33,33,33,33,33,33,0
1966,40,40,40,40,40,40,40,40,0
1967,62,62,62,62,62,62,62,62,0
1968,547,547,547,547,547,547,547,547,0
1969,3836,3836,3835,3836,3836,3836,3836,3836,0
1970,3853,3853,3852,3853,3853,3853,3853,3853,0
1971,3821,3821,3820,3821,3821,3821,3821,3821,0
1972,3880,3880,3880,3880,3880,3880,3880,3880,0
1973,4139,4139,4139,4139,4139,4139,4138,4139,0


In [62]:
all_death[all_death.CATEGORY_CODE=='COVID-19  '].drop_duplicates('MED_EDU_NBR').groupby(['YEAR']).count()

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,CATEGORY_CODE
YEAR,,,,,,,,,
2020,142,142,142,142,142,142,142,142,142


In [55]:
all_death[all_death['CATEGORY_CODE']=='SUICIDE']

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,CATEGORY_CODE,YEAR


In [83]:
DEATH.groupby('CATEGORY_CODE').count()

,PARTY_ID
CATEGORY_CODE,
CAR-ACCID,10
COVID-19,147
HOMICIDE,10
NATRL-CAUS,77
SUICIDE,13


In [52]:
all_death = all_death.fillna('None')

In [58]:
list(all_death[all_death.CATEGORY_CODE!='None']['CATEGORY_CODE'])

['COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'HOMICIDE  ',
 'HOMICIDE  ',
 'CAR-ACCID ',
 'SUICIDE   ',
 'NATRL-CAUS',
 'COVID-19  ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'SUICIDE   ',
 'COVID-19  ',
 'NATRL-CAUS',
 'SUICIDE   ',
 'SUICIDE   ',
 'SUICIDE   ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'COVID-19  ',
 'CAR-ACCID ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'HOMICIDE  ',
 'HOMICIDE  ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'COVID-19  ',
 'SUICIDE   ',
 'HOMICIDE  ',
 'HOMICIDE  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'COVID-19  ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'COVID-19  ',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'NATRL-CAUS',
 'COVID-19  ',
 'COVID-19

In [72]:
all_death[all_death.BIRTH_CITY_NM=='New York City']

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,BIRTH_CITY_NM,CATEGORY_CODE
382154,65401870142,2743626,1952-01-10,2014-12-17,N,Howland,Rachel,N Rachel Howland,New York City,NaN
382155,65401870142,2743626,1952-01-10,2014-12-17,N,Howland,Rachel,N Rachel Howland MD,New York City,NaN
382156,65401870142,2743626,1952-01-10,2014-12-17,Nanci,Howland,Rachel,Nanci Rachel Howland,New York City,NaN
382157,65401870142,2743626,1952-01-10,2014-12-17,Nanci,Howland,Rachel,Nanci Rachel Howland MD,New York City,NaN
417426,30810830120,4589007,1945-12-13,2011-01-01,Anibal,Giannoni Molina,A,Anibal A Giannoni Molina MD,New York City,NaN
417427,30810830120,4589007,1945-12-13,2011-01-01,Anibal,Giannoni Molina,A.,Anibal A. Giannoni Molina MD,New York City,NaN
417428,30810830120,4589007,1945-12-13,2011-01-01,Anibal,Giannoni,Antonio,Anibal Antonio Giannoni MD,New York City,NaN


In [73]:
all_death['DEATH_DT'] = pd.to_datetime(all_death.DEATH_DT)
all_death['BIRTH_DT'] = pd.to_datetime(all_death.BIRTH_DT)

In [77]:
[(all_death.DEATH_DT - all_death.BIRTH_DT)]

[0        17008 days
 1        31183 days
 2        31183 days
 3        31114 days
 4        30609 days
             ...    
 417783   10197 days
 417784    9164 days
 417785    8597 days
 417786    8992 days
 417787    8782 days
 Length: 417788, dtype: timedelta64[ns]]

In [80]:
all_death['BIRTH_YEAR']=[x.year for x in all_death.BIRTH_DT]
all_death['DEATH_YEAR']=[x.year for x in all_death.DEATH_DT]
all_death['AGE'] = [all_death.DEATH_YEAR - all_death.BIRTH_YEAR]

ValueError: Length of values does not match length of index

In [82]:
DEATH.groupby('CATEGORY_CODE')

In [86]:
all_death = all_death.fillna('None')

In [87]:
ages = []
for row in all_death.itertuples():
    if row.DEATH_YEAR!='None' and row.BIRTH_YEAR != 'None':
        age = row.DEATH_YEAR - row.BIRTH_YEAR
    else:
        age = -1
    ages.append(age)

In [88]:
all_death['AGE'] = ages

In [95]:
all_death.sort_values('AGE').drop_duplicates('MED_EDU_NBR').groupby('AGE').count()[3:30]

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,BIRTH_CITY_NM,CATEGORY_CODE,BIRTH_YEAR,DEATH_YEAR
AGE,,,,,,,,,,,,
19.0,1,1,1,1,1,1,1,1,1,1,1,1
21.0,7,7,7,7,7,7,7,7,7,7,7,7
22.0,27,27,27,27,27,27,27,27,27,27,27,27
23.0,70,70,70,70,70,70,70,70,70,70,70,70
24.0,99,99,99,99,99,99,99,99,99,99,99,99
25.0,130,130,130,130,130,130,130,130,130,130,130,130
26.0,183,183,183,183,183,183,183,183,183,183,183,183
27.0,212,212,212,212,212,212,212,212,212,212,212,212
28.0,246,246,246,246,246,246,246,246,246,246,246,246


In [111]:
all_death.drop_duplicates('MED_EDU_NBR').sort_values('AGE')[96:]

,MED_EDU_NBR,PARTY_ID,BIRTH_DT,DEATH_DT,FIRST_NM,LAST_NM,MIDDLE_NM,LABEL_NM,BIRTH_CITY_NM,CATEGORY_CODE,BIRTH_YEAR,DEATH_YEAR,AGE
417679,01720160575,6414003,1993-06-08 00:00:00,2012-11-07,Christopher,Hrvoj,Rafael,Christopher Rafael Hrvoj,None,None,1993,2012,19.0
417417,04812101965,4577589,1985-04-21 00:00:00,2006-12-27,Paul,Wikholm,Andrew,Paul Andrew Wikholm,ORANGE,None,1985,2006,21.0
258316,04102930091,2405718,1969-05-10 00:00:00,1990-02-13,Rajesh,Babbar,Kumar,Rajesh Kumar Babbar,KINGSTON,None,1969,1990,21.0
31148,01201761049,1883022,1951-02-11 00:00:00,1972-12-28,Richard,Mixon,Timothy,Richard Timothy Mixon,AUGUSTA,None,1951,1972,21.0
415919,02802030552,2939258,1979-01-15 00:00:00,2000-01-18,Adam,Elkhishin,,Adam Elkhishin,BALLARAT,None,1979,2000,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48533,01201280041,1931843,1898-02-01 00:00:00,2012-04-01,Leila,Daughtry-Denmark,A,Leila A Daughtry-Denmark MD,PORTAL,None,1898,2012,114.0
175304,03510920014,2204942,1865-12-10 00:00:00,1981-12-01,Alfred,Mooney,H,Alfred H Mooney MD,USA,None,1865,1981,116.0
280798,10000000766,2468380,1803-12-25 00:00:00,1973-08-31,Martin,Lichter,Herman,Martin Herman Lichter MD,BAKU,None,1803,1973,170.0
341586,10000000031,2625152,1802-01-22 00:00:00,1973-02-12,Ralph,Alford,J,Ralph J Alford MD,EAST DURHAM,None,1802,1973,171.0


In [1]:
all_death

NameError: name 'all_death' is not defined

In [54]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20200718.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
ppd[(ppd.FIRST_NAME=='GINA')&(ppd.LAST_NAME=='MADOLE')][['MAILING_NAME','BIRTH_YEAR','ME']]

,MAILING_NAME,BIRTH_YEAR,ME


In [68]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20200905.dos', sep='|', encoding='ISO-8859-1')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
ppd_path = '../../Data/PPD/PhysicianProfessionalDataFile_20200905'

In [71]:
cols = [
        'ME',
        'RECORD_ID',
        'UPDATE_TYPE',
        'ADDRESS_TYPE',
        'MAILING_NAME',
        'LAST_NAME',
        'FIRST_NAME',
        'MIDDLE_NAME',
        'SUFFIX',
        'MAILING_LINE_1',
        'MAILING_LINE_2',
        'CITY',
        'STATE',
        'ZIP',
        'SECTOR',
        'CARRIER_ROUTE',
        'ADDRESS_UNDELIVERABLE_FLAG',
        'FIPS_COUNTY',
        'FIPS_STATE',
        'PRINTER_CONTROL_CODE',
        'PC_ZIP',
        'PC_SECTOR',
        'DELIVERY_POINT_CODE',
        'CHECK_DIGIT',
        'PRINTER_CONTROL_CODE_2',
        'REGION',
        'DIVISION',
        'GROUP',
        'TRACT',
        'SUFFIX_CENSUS',
        'BLOCK_GROUP',
        'MSA_POPULATION_SIZE',
        'MICRO_METRO_IND',
        'CBSA',
        'CBSA_DIV_IND',
        'MD_DO_CODE',
        'BIRTH_YEAR',
        'BIRTH_CITY',
        'BIRTH_STATE',
        'BIRTH_COUNTRY',
        'GENDER',
        'TELEPHONE_NUMBER',
        'PRESUMED_DEAD_FLAG',
        'FAX_NUMBER',
        'TOP_CD',
        'PE_CD',
        'PRIM_SPEC_CD',
        'SEC_SPEC_CD',
        'MPA_CD',
        'PRA_RECIPIENT',
        'PRA_EXP_DT',
        'GME_CONF_FLG',
        'FROM_DT',
        'TO_DT',
        'YEAR_IN_PROGRAM',
        'POST_GRADUATE_YEAR',
        'GME_SPEC_1',
        'GME_SPEC_2',
        'TRAINING_TYPE',
        'GME_INST_STATE',
        'GME_INST_ID',
        'MEDSCHOOL_STATE',
        'MEDSCHOOL_ID',
        'MEDSCHOOL_GRAD_YEAR',
        'NO_CONTACT_IND',
        'NO_WEB_FLAG',
        'PDRP_FLAG',
        'PDRP_START_DT',
        'POLO_MAILING_LINE_1',
        'POLO_MAILING_LINE_2',
        'POLO_CITY',
        'POLO_STATE',
        'POLO_ZIP',
        'POLO_SECTOR',
        'POLO_CARRIER_ROUTE',
        'MOST_RECENT_FORMER_LAST_NAME',
        'MOST_RECENT_FORMER_MIDDLE_NAME',
        'MOST_RECENT_FORMER_FIRST_NAME',
        'NEXT_MOST_RECENT_FORMER_LAST',
        'NEXT_MOST_RECENT_FORMER_MIDDLE',
        'NEXT_MOST_RECENT_FORMER_FIRST'
    ]

# read the file and create a DataFrame
ppd = pd.read_csv(ppd_path, names=cols, sep='|', encoding='IBM437', index_col=False, dtype=object)

In [73]:
ppd.to_csv('../../Data/PPD/ppd_data_20200905.csv', index=False)

In [77]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
today_date = datetime.today()
scorecard_month = (today_date - relativedelta(months=1)).strftime("%b" " " "%Y")
last_month = (today_date - relativedelta(months=2)).strftime("%b" " " "%Y")
last_year = (today_date - relativedelta(months=2, years=1)).strftime("%b" " " "%Y")
two_year = (today_date - relativedelta(months=2, years=2)).strftime("%b" " " "%Y")
print(f'This month is {scorecard_month}')
print(f'Last month was {last_month}')
print(f'Last year was {last_year}')

This month is Aug 2020
Last month was Jul 2020
Last year was Jul 2019
